In [1]:
from astropy.io import fits
import astropy.wcs 
from astropy import table
import sep
import numpy as np
from collections import OrderedDict

import astropyp
print astropyp.version.version

%matplotlib inline

img_path = '/media/data-beta/users/fmooleka/decam/std_resamp/442493.resamp.fits'
img = fits.open(img_path)
dqmask = fits.open(img_path.replace('.fits', '.dqmask.fits'))
frame = 2
img_data = img[frame].data.byteswap(True).newbyteorder()
dqmask_data = dqmask[frame].data.byteswap(True).newbyteorder()
img[frame].header['CTYPE1'] = 'RA---TPV'
img[frame].header['CTYPE2'] = 'DEC--TPV'
header = img[frame].header
wcs = astropy.wcs.WCS(img[frame].header)

0.0.dev221


In [6]:
conv_filter = np.array([
    [0.030531, 0.065238, 0.112208, 0.155356, 0.173152, 0.155356, 0.112208, 0.065238, 0.030531],
    [0.065238, 0.139399, 0.239763, 0.331961, 0.369987, 0.331961, 0.239763, 0.139399, 0.065238],
    [0.112208, 0.239763, 0.412386, 0.570963, 0.636368, 0.570963, 0.412386, 0.239763, 0.112208],
    [0.155356, 0.331961, 0.570963, 0.790520, 0.881075, 0.790520, 0.570963, 0.331961, 0.155356],
    [0.173152, 0.369987, 0.636368, 0.881075, 0.982004, 0.881075, 0.636368, 0.369987, 0.173152],
    [0.155356, 0.331961, 0.570963, 0.790520, 0.881075, 0.790520, 0.570963, 0.331961, 0.155356],
    [0.112208, 0.239763, 0.412386, 0.570963, 0.636368, 0.570963, 0.412386, 0.239763, 0.112208],
    [0.065238, 0.139399, 0.239763, 0.331961, 0.369987, 0.331961, 0.239763, 0.139399, 0.065238],
    [0.030531, 0.065238, 0.112208, 0.155356, 0.173152, 0.155356, 0.112208, 0.065238, 0.030531]
])
sex_params = {
    'extract': {
        #'thresh': 1.5,# *bkg.globalrms,
        #'err':,
        #'minarea': 5, # default
        'conv': conv_filter,
        #'deblend_nthresh': 32, #default
        'deblend_cont': 0.001,
        #'clean': True, #default
        #'clean_param': 1 #default
    },
    'kron_k': 2.5,
    'kron_min_radius': 3.5,
    'filter': conv_filter,
    'thresh': 1.5
}

ccd = astropyp.phot.phot.SingleImage(header, img_data, dqmask_data, 
    wcs=wcs, gain=4., exptime=30, aper_radius=8)
ccd.detect_sources(sex_params)
ccd.select_psf_sources(1000,1000)
psf_array = ccd.create_psf()
#ccd.show_psf()
#psf_groups = ccd.create_psf_groups(verbose=True)
good_idx = (ccd.catalog.sources['flags']==0) & (ccd.catalog.sources['peak']>100)
psf_flux = ccd.perform_psf_photometry(indices=good_idx)

INFO:astropyp.phot.psf:Total sources: 473
INFO:astropyp.phot.psf:Sources with low flux: 89
INFO:astropyp.phot.psf:Sources with low amplitude: 410
INFO:astropyp.phot.psf:Sources with bad pixels: 99
INFO:astropyp.phot.psf:Elliptical sources: 50
INFO:astropyp.phot.psf:Source with close neighbors: 56
INFO:astropyp.phot.psf:Sources near an edge: 1
INFO:astropyp.phot.psf:Sources after cuts: 18


In [3]:
groups = [g for g,grp in psf_groups.items() if isinstance(grp, astropyp.phot.psf.GroupPSF)]
groups

[45, 130, 137, 167, 171, 197, 199, 241, 346, 355, 389, 432, 437]

In [4]:
good_idx = (ccd.catalog.sources['flags']==0) & (ccd.catalog.sources['peak']>100)
psf_flux = ccd.perform_psf_photometry(group_sources=False, indices=good_idx)

In [7]:
ccd.catalog['psf_mag', 'psf_mag_err'][good_idx]

psf_mag,psf_mag_err
float64,float64
-7.44944880759,0.00634393899648
-5.96538389255,0.0223916677359
-4.70995789158,0.0691719721804
-7.90640462773,0.00443451836823
-5.11959038922,0.0477232165822
-7.49595008859,0.00611187952582
-4.70346048467,0.0695815942752
-5.69552112739,0.0284543777232
-5.69552112739,0.0284543777232
